## Welcome to the HWComponents Tutorial!

This tutorial will guide you through the process of using HWComponents to estimate the
energy and area components in a hardware design.

To start, let's import the necessary modules.

In [1]:
import logging
logging.getLogger().setLevel(logging.WARNING)
import hwcomponents as hwc

### Listing available models

HWComponents uses a suite of models to estimate the energy and area of components.
You can list the available models from the shell with the "hwcomponents --list"
function. Let's list the available models.

In [2]:
# The output is too long to display here, so we'll just display the first 50 lines. 
! hwcomponents --list 2> /dev/null | head -50



Supported Components:
adc(n_bits, tech_node, throughput, n_adcs=1) from class ADC 
	activate()
	convert()
	drive()
	leak(time_period)
	read()
	sample()
	leak(time_period)
adder(tech_node, width=32) from class AladdinAdder 
	add()
	leak(time_period)
	read()
	write()
	leak(time_period)
adder(tech_node, global_cycle_seconds, n_bits) from class Adder 
	add()
	compute()
	convert()
	leak(time_period)
	read()
	shift_add()
	update()
	write()
	leak(time_period)
adder_tree(tech_node, global_cycle_seconds, n_bits, n_adder_tree_inputs) from class AdderTree 
	add()
	compute()
	convert()
	leak(time_period)
	read()
	shift_add()
	update()
	write()
	leak(time_period)
aladdin_adder: alias for AladdinAdder from class adder
aladdin_comparator: alias for AladdinComparator from class comparator
aladdin_counter: alias for AladdinCounter from class counter
aladdin_multiplier: alias for AladdinMultiplier from class intmultiplier
aladdin_register: alias for AladdinRegister from class register
array_adc: alias

### Using Models from HWComponents

There are three ways use models.

1. Import the model from a module and use it directly.
2. Ask hwcomponents to select the best model for a given component.
3. Ask hwcomponents for specific properties of a component.

All units are in base units; Joules, watts, meters, seconds, etc. 

In [3]:
# Method 1: Import the model from a module and use it directly.
from hwcomponents_cacti import SRAM
sram = SRAM(
    tech_node=40e-9, # 40nm
    width=64,
    depth=1024,
)
print(f"SRAM read energy is {sram.read():.2e}J. Area is {sram.area:.2e}m^2. Leak power is {sram.leak_power:.2e}W")


# Method 2: Ask hwcomponents to select the best model for a given component.
model = hwc.get_model(
    component_name="SRAM", # These are NOT case sensitive.
    component_attributes={
        "tech_node": 40e-9, # 40nm
        "width": 64,
        "depth": 1024
    },
    required_actions=["read"]
)
print(f'Read energy is {model.read():.2e}J. Area is {model.area:.2e}m^2. Leak power is {model.leak_power:.2e}W')

# Method 3: Ask for specific properties from hwcomponents
attributes = {
    "tech_node": 40e-9, # 40nm
    "width": 64,
    "depth": 1024
}

read_energy = hwc.get_energy(
    component_name="SRAM",
    component_attributes=attributes,
    action_name="read",
    action_arguments={}
)
area = hwc.get_area(
    component_name="SRAM",
    component_attributes=attributes,
)
leak_power = hwc.get_leak_power(
    component_name="SRAM",
    component_attributes=attributes,
)
print(f'Read energy is {read_energy:.2e}J. Area is {area:.2e}m^2. Leak power is {leak_power:.2e}W')

SRAM read energy is 5.10e+06J. Area is 3.31e-07m^2. Leak power is 2.13e-08W
Read energy is 5.10e+06J. Area is 3.31e-07m^2. Leak power is 2.13e-08W
Read energy is 5.10e+06J. Area is 3.31e-07m^2. Leak power is 2.13e-08W


### Creating Custom Models

Models can be created by subclassing the `EnergyAreaModel` class. Models
estimate the energy, area, and leakage power of a component. Each model requires the
following:
- `component_name`: The name of the component. This may also be a list of components if
  multiple aliases are used.
- `priority_0_to_100`: The percent accuracy of the model. This is used to
  break ties if multiple models support a given query.
- A call to `super().__init__(area, leak_power)`. This is used to initialize the
  model and set the area and leakage power.

Models can also have actions. Actions are functions that return an energy of a
specific action. For the TernaryMAC model, we have an action called `mac` that
returns the energy of a ternary MAC operation. The `@actionDynamicEnergy` decorator
makes this function visible as an action. The function should return an energy in
Joules.

Models can also be scaled to support a range of different parameters. For example,
the TernaryMAC model can be scaled to support a range of different technology nodes.
This is done by calling the `self.scale` function in the `__init__` method of the
model. The `self.scale` function takes the following arguments:
- `parameter_name`: The name of the parameter to scale.
- `parameter_value`: The value of the parameter to scale.
- `reference_value`: The reference value of the parameter.
- `area_scaling_function`: The scaling function to use for area. Use `None` if no
  scaling should be done.
- `energy_scaling_function`: The scaling function to use for dynamic energy. Use `None` if no
  scaling should be done.
- `leak_scaling_function`: The scaling function to use for leakage power. Use `None` if
  no scaling should be done.

Many different scaling functions are defined and available in `hwcomponents.scaling`.

In [4]:
from hwcomponents import EnergyAreaModel, actionDynamicEnergy
from hwcomponents.scaling import tech_node_area, tech_node_energy, tech_node_leak

class TernaryMAC(EnergyAreaModel):
    component_name: str | list[str] = 'TernaryMAC'
    """ Name of the component. Must be a string or list/tuple of strings. """

    priority = 0.8
    """
    Priority determines which model is used when multiple models are available for a
    given component. Higher priority models are used first. Must be a number between 0
    and 1.
    """

    def __init__(self, accum_datawidth: int, tech_node: int):
        # Provide an area and leakage power for the component. All units are in 
        # standard units without any prefixes (Joules, Watts, meters, etc.).
        super().__init__(
            area=5e-12 * accum_datawidth, 
            leak_power=1e-3 * accum_datawidth
        )

        # The following scales the tech_node to the given tech_node node from 40nm. 
        # The scaling functions for area, energy, and leakage are defined in
        # hwcomponents.scaling. The energy scalingw will affect the functions decorated
        # with @actionDynamicEnergy.
        self.tech_node = self.scale(
            "tech_node",
            tech_node,
            40e-9,
            tech_node_area,
            tech_node_energy,
            tech_node_leak,
        )
        self.accum_datawidth = accum_datawidth

        # Raising an error says that this model can't estimate and other models instead
        # should be used instead. Good error messages are essential for users debugging
        # their designs.
        assert 4 <= accum_datawidth <= 8, \
            f'Accumulation datawidth {accum_datawidth} outside supported ' \
            f'range [4, 8]!'

    # The actionDynamicEnergy decorator makes this function visible as an action. The
    # function should return an energy in Joules.
    @actionDynamicEnergy
    def mac(self, clock_gated: bool = False) -> float:
        self.logger.info(f'TernaryMAC Model is estimating '
                         f'energy for mac_random.')
        if clock_gated:
            return 0.0
        return 0.002e-12 * (self.accum_datawidth + 0.25)

mac = TernaryMAC(accum_datawidth=8, tech_node=16e-9) # Scale the TernaryMAC to 16nm
print(f'TernaryMAC energy is {mac.mac():.2e}J. Area is {mac.area:.2e}m^2. Leak power is {mac.leak_power:.2e}W')

TernaryMAC energy is 5.37e-15J. Area is 4.00e-11m^2. Leak power is 8.00e-03W


That's all! You can now use the TernaryMAC model to estimate the energy and area of
a ternary MAC operation.

That's the end of the tutorial. Please submit any issues or feature requests to the
HWComponents GitHub repository.

Thank you for using HWComponents!